<a href="https://colab.research.google.com/github/CE339/CES-S24/blob/main/Module02/RiverpolAbstractModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initializing Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

**Installing Pyomo and a solver**. Instead of installing glpk, this time we will install COIN-OR CBC. \\
COIN-OR CBC is a multi-threaded open-source Coin-or branch and cut **mixed-integer linear programming solver**. CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [ ]:
!pip install pyomo
#!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

**Importing Pyomo and CBC Solver**

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

**Building the abstract model**

Now that we are begining to create more general models we will learn how to work with abstract models. The excercise we did before where we tried to generalize the model was a preparation for understanding **abstract models**. \\
Abstract models are just defined by symbols and rules. You can not use the argument expr when defining constraints and objetive for abstract models.

The idea is to create a general model that would be used to solve multiple **instances** fo the same optimization problem. In other words we could solve the same problem for different parameters.

In [ ]:
from math import inf

model = AbstractModel()

model.I = Set() # Set of Factories
model.J = Set() # Set of Pollutants

# Parameters Definition
# ci - Cost to process waste from factory
model.c    = Param(model.I, within=PositiveReals)
# rij - Rate of reduction in pollutant type if emission is treated at factory
model.r    = Param(model.I, model.J, within=NonNegativeReals)
# Sj - State requirement to reduce pollutants
model.S = Param(model.J)

# wi - Processed waste from Factory
model.w = Var(model.I, within=NonNegativeReals)

# Minimize the cost to reduce pollutants
def cost_rule(model):
    return sum(model.c[i]*model.w[i] for i in model.I)
model.cost = Objective(rule=cost_rule)

# Satisfy the state's requeriments on pollutant reduction
def pollutant_rule(model, j):
    return sum(model.r[i,j]*model.w[i] for i in model.I) >= model.S[j]
model.pollutant = Constraint(model.J, rule=pollutant_rule)

**Creating the model instance** - using data contained in the data file riverpol.dat

In [ ]:
instance = model.create_instance('riverpol.dat')
instance.pprint()

**Optimizing the model instance**

In [ ]:
results = opt.solve(instance, tee=True)

**Display Results**

In [ ]:
print(instance.display())

**Creating another Instance of the Model**

In [ ]:
instance2 = model.create_instance('riverpol3factories.dat')
instance2.pprint()

**Optimizing Instance 2 and Displaying Results**

In [ ]:
results2 = opt.solve(instance2, tee=True)
print(instance2.display())